# Predicting heart disease using machine learning

This notebook looks into using various Python based machine learning and data science libraries in an an attempt to build a machine learning model capable of predicitng whether someone has heart disease or not based on their medical attributes.

We're going to take the following approach:
1. Problem definition
2. Data
3. Evaluate
4. Features
5. Modelling
6. Experimentation

## 1. Problem Definition
In a statement, 
> Given clinical parameters about a patient, can we predict whether or not they have heart disease?

## 2. Data

The original data came from the Cleveland data from UCI Machine Learning Repository. 

There is also a version of it available on Kaggle.

## 3. Evaluation 
> If we can reach 95% accuracy at prediciting whether or not a patient has heart disease during the proof of concept, we'll purse the project.

## 4. Features

This is where you get information about each of the features in your data.

**create data dictionary**
